Get it done:
+ create a series of clusters and pick the one that best represents the data
+ perform (unsupervised) feature generation and selection
+ use the features to build models to classify texts by author
+ on holdout group, does clustering perform as you'd expect? is model performance consistent?
+ how do clustering and modeling compare for classifying texts? what are the advantages of each? why would you want to use one over the other?

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import sklearn
import spacy
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

<h1><center>Load Data And Use Vectorizer to Generate Features</center></h1>

In [35]:
from nltk.corpus import inaugural

In [36]:
inaugural.fileids()

['1789-Washington.txt',
 '1793-Washington.txt',
 '1797-Adams.txt',
 '1801-Jefferson.txt',
 '1805-Jefferson.txt',
 '1809-Madison.txt',
 '1813-Madison.txt',
 '1817-Monroe.txt',
 '1821-Monroe.txt',
 '1825-Adams.txt',
 '1829-Jackson.txt',
 '1833-Jackson.txt',
 '1837-VanBuren.txt',
 '1841-Harrison.txt',
 '1845-Polk.txt',
 '1849-Taylor.txt',
 '1853-Pierce.txt',
 '1857-Buchanan.txt',
 '1861-Lincoln.txt',
 '1865-Lincoln.txt',
 '1869-Grant.txt',
 '1873-Grant.txt',
 '1877-Hayes.txt',
 '1881-Garfield.txt',
 '1885-Cleveland.txt',
 '1889-Harrison.txt',
 '1893-Cleveland.txt',
 '1897-McKinley.txt',
 '1901-McKinley.txt',
 '1905-Roosevelt.txt',
 '1909-Taft.txt',
 '1913-Wilson.txt',
 '1917-Wilson.txt',
 '1921-Harding.txt',
 '1925-Coolidge.txt',
 '1929-Hoover.txt',
 '1933-Roosevelt.txt',
 '1937-Roosevelt.txt',
 '1941-Roosevelt.txt',
 '1945-Roosevelt.txt',
 '1949-Truman.txt',
 '1953-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1965-Johnson.txt',
 '1969-Nixon.txt',
 '1973-Nixon.txt',
 '1

In [41]:
Obama = inaugural.sents("2009-Obama.txt")
Clinton = inaugural.sents("1993-Clinton.txt")
Clinton1 = inaugural.sents("1997-Clinton.txt")
Reagan = inaugural.sents("1981-Reagan.txt")
Nixon = inaugural.sents("1969-Nixon.txt")
Eisenhower = inaugural.sents("1957-Eisenhower.txt")
Eisenhower1 = inaugural.sents("1953-Eisenhower.txt")
McKinley = inaugural.sents("1897-McKinley.txt")
McKinley1 = inaugural.sents("1901-McKinley.txt")
Wilson = inaugural.sents("1917-Wilson.txt")
Wilson1 = inaugural.sents("1913-Wilson.txt")
Roosevelt = inaugural.sents("1933-Roosevelt.txt")
Roosevelt1 = inaugural.sents("1937-Roosevelt.txt")
Cleveland = inaugural.sents("1893-Cleveland.txt")
Cleveland1 = inaugural.sents("1885-Cleveland.txt")
Garfield = inaugural.sents("1881-Garfield.txt")

In [42]:
Obama[1]

['I',
 'stand',
 'here',
 'today',
 'humbled',
 'by',
 'the',
 'task',
 'before',
 'us',
 ',',
 'grateful',
 'for',
 'the',
 'trust',
 'you',
 'have',
 'bestowed',
 ',',
 'mindful',
 'of',
 'the',
 'sacrifices',
 'borne',
 'by',
 'our',
 'ancestors',
 '.']

In [48]:
Obama[-1]

['And', 'God', 'bless', 'the', 'United', 'States', 'of', 'America', '.']

In [43]:
Clinton[1]

['This',
 'ceremony',
 'is',
 'held',
 'in',
 'the',
 'depth',
 'of',
 'winter',
 ',',
 'but',
 'by',
 'the',
 'words',
 'we',
 'speak',
 'and',
 'the',
 'faces',
 'we',
 'show',
 'the',
 'world',
 ',',
 'we',
 'force',
 'the',
 'spring',
 '.']

In [91]:
inaugural_sents = []

for sentence in Obama:
    inaugural_sents.append(" ".join(sentence))

print("Number of Obama sentences", len(inaugural_sents))
    
for sentence in Clinton:
    inaugural_sents.append(" ".join(sentence))
    
for sentence in Clinton1:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Clinton:", len(inaugural_sents))

for sentence in Reagan:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Reagan:", len(inaugural_sents))

for sentence in Nixon:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Nixon:", len(inaugural_sents))

for sentence in Eisenhower:
    inaugural_sents.append(" ".join(sentence))
    
for sentence in Eisenhower1:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Eisenhower:", len(inaugural_sents))

for sentence in Garfield:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Garfield:", len(inaugural_sents))

for sentence in McKinley:
    inaugural_sents.append(" ".join(sentence))
    
for sentence in McKinley1:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after McKinley:", len(inaugural_sents))

for sentence in Wilson:
    inaugural_sents.append(" ".join(sentence))
    
for sentence in Wilson1:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Wilson:", len(inaugural_sents))

for sentence in Roosevelt:
    inaugural_sents.append(" ".join(sentence))
    
for sentence in Roosevelt1:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Roosevelt:", len(inaugural_sents))

for sentence in Cleveland:
    inaugural_sents.append(" ".join(sentence))
    
for sentence in Cleveland1:
    inaugural_sents.append(" ".join(sentence))
    
print("Number of sentences after Cleveland:", len(inaugural_sents))

Number of Obama sentences 112
Number of sentences after Clinton: 305
Number of sentences after Reagan: 432
Number of sentences after Nixon: 538
Number of sentences after Eisenhower: 753
Number of sentences after Garfield: 865
Number of sentences after McKinley: 1095
Number of sentences after Wilson: 1223
Number of sentences after Roosevelt: 1404
Number of sentences after Cleveland: 1506


In [95]:
inaugural_sents[113]

'This ceremony is held in the depth of winter , but by the words we speak and the faces we show the world , we force the spring .'

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
vect_inaugural_sents=vectorizer.fit_transform(inaugural_sents)
print("Number of features: %d" % vect_inaugural_sents.get_shape()[1])

Number of features: 2053


In [103]:
vect_inaugural_sents_csr = vect_inaugural_sents.tocsr()

n = vect_inaugural_sents_csr.shape[0]

tfidf_bypara = [{} for _ in range(0,n)]

terms = vectorizer.get_feature_names()

for i, j in zip(*vect_inaugural_sents_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = vect_inaugural_sents_csr[i, j]

In [104]:
word_vectors = pd.DataFrame(tfidf_bypara)

word_vectors.head()

,000,1787,1890,1897,19th,20th,21st,abandon,abandoned,abandonment,...,written,wrong,wronged,year,years,yes,yield,young,youth,zeal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
word_vectors.sum(axis=0)

000                1.279445
1787               0.831377
1890               0.561347
1897               0.772842
19th               0.828161
20th               0.905539
21st               1.412740
abandon            0.751256
abandoned          1.646255
abandonment        0.513629
abiding            1.649074
ability            2.511144
able               0.703075
abroad             1.430666
absence            0.776563
abundance          0.876721
accept             2.472588
accepted           0.831240
accomplished       1.972054
accomplishment     0.913701
account            1.417318
achieve            2.575203
achievement        2.488357
achievements       1.431531
acquiescence       0.595798
act                5.625846
action             7.335914
actions            0.692628
active             0.711026
activities         0.922531
                    ...    
winds              1.433305
winter             1.057247
wisdom             3.486752
wise               1.167973
wisely             2

In [106]:
word_vectors = word_vectors.fillna(0)

In [88]:
from sklearn.preprocessing import normalize

word_vectors = normalize(word_vectors)

In [107]:
word_vectors.shape

(1506, 2053)

In [108]:
word_vectors.head()

,000,1787,1890,1897,19th,20th,21st,abandon,abandoned,abandonment,...,written,wrong,wronged,year,years,yes,yield,young,youth,zeal
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
word_vectors["speech_sentence"] = inaugural_sents

In [111]:
word_vectors.head()

,000,1787,1890,1897,19th,20th,21st,abandon,abandoned,abandonment,...,wrong,wronged,year,years,yes,yield,young,youth,zeal,speech_sentence
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,My fellow citizens :
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I stand here today humbled by the task before ...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I thank President Bush for his service to our ...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Forty - four Americans have now taken the pres...
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,The words have been spoken during rising tides...


In [112]:
speech_source = []
for i in range(0,len(Obama)):
    speech_source.append("Obama")
for i in range(0,len(Clinton)+len(Clinton1)):
    speech_source.append("Clinton")
for i in range(0,len(Reagan)):
    speech_source.append("Reagan")
for i in range(0,len(Nixon)):
    speech_source.append("Nixon")
for i in range(0,len(Garfield)):
    speech_source.append("Garfield")
for i in range(0,len(McKinley)+len(McKinley1)):
    speech_source.append("McKinley")
for i in range(0,len(Eisenhower)+len(Eisenhower1)):
    speech_source.append("Eisenhower")
for i in range(0,len(Wilson)+len(Wilson1)):
    speech_source.append("Wilson")   
for i in range(0,len(Roosevelt)+len(Roosevelt1)):
    speech_source.append("Roosevelt")    
for i in range(0,len(Cleveland)+len(Cleveland1)):
    speech_source.append("Cleveland")
    
print(len(speech_source))

1506


In [113]:
word_vectors["speech_source"] = speech_source

In [114]:
word_vectors.shape

(1506, 2055)

In [115]:
Y = word_vectors["speech_source"]
X = np.array(word_vectors.loc[:, ~word_vectors.columns.isin(["speech_sentence", "speech_source"])])
print(X.shape)
print(Y.shape)

(1506, 2053)
(1506,)


In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

<h1><center>Create a Series of Clusters</center></h1>

### k-Means

In [145]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances

In [146]:
kmeans = KMeans(n_clusters=10)
kmeans = kmeans.fit(X_train)
kmlabels = kmeans.predict(X_train)

In [157]:
from collections import Counter

Counter(kmlabels)

Counter({0: 579,
         9: 84,
         5: 61,
         4: 125,
         7: 76,
         3: 83,
         6: 21,
         1: 31,
         2: 32,
         8: 37})

In [152]:
534/1129

0.47298494242692646

### Meanshift

In [134]:
from sklearn.cluster import MeanShift, estimate_bandwidth

quantiles = [.1, .2, .3, .4, .5, .6, .65, .75, .9]

for q in quantiles:
    bandwidth = estimate_bandwidth(X_train, quantile=q, n_samples=100)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms = ms.fit(X_train)
    mslabels = ms.labels_
    print(q)
    print(len(ms.cluster_centers_))

0.1
1
0.2
1
0.3
1
0.4
1
0.5
1
0.6
1
0.65
1
0.75
1
0.9
1


In [135]:
ms = MeanShift()
ms = ms.fit(X_train)
mslabels = ms.labels_

In [136]:
Counter(mslabels)

Counter({0: 1129})

### Spectral Clustering

In [137]:
from sklearn.cluster import SpectralClustering

sc = SpectralClustering(n_clusters=10)
sc = sc.fit(X_train)
sclabels = sc.labels_

In [138]:
Counter(sclabels)

Counter({0: 707, 6: 39, 8: 38, 9: 66, 4: 109, 5: 94, 1: 18, 7: 46, 3: 9, 2: 3})

### Digging into the different clustering methods

In [143]:
len(sclabels)

1129

In [144]:
len(y_train)

1129

In [147]:
cluster_labels_df = pd.DataFrame()

cluster_labels_df["speech_source"] = y_train
cluster_labels_df["km_clusters"] = kmlabels
cluster_labels_df["ms_clusters"] = mslabels
cluster_labels_df["sc_clusters"] = sclabels

In [151]:
Counter(cluster_labels_df[cluster_labels_df["speech_source"] == "Obama"]["km_clusters"])

Counter({0: 48, 2: 3, 4: 4, 8: 5, 1: 3, 9: 8, 6: 1, 7: 3, 5: 3})

In [153]:
Counter(cluster_labels_df[cluster_labels_df["speech_source"] == "Cleveland"]["km_clusters"])

Counter({0: 25, 3: 18, 4: 4, 7: 12, 5: 12, 8: 2, 1: 3, 2: 2, 9: 2})

In [154]:
Counter(cluster_labels_df[cluster_labels_df["speech_source"] == "Garfield"]["km_clusters"])

Counter({4: 12, 0: 44, 7: 5, 2: 2, 9: 6, 6: 4, 5: 3, 1: 2, 3: 1, 8: 2})

In [155]:
(Counter(cluster_labels_df[cluster_labels_df["km_clusters"] == 8]["speech_source"]))

Counter({'Reagan': 3,
         'Obama': 5,
         'Cleveland': 2,
         'McKinley': 14,
         'Eisenhower': 3,
         'Clinton': 2,
         'Roosevelt': 2,
         'Nixon': 2,
         'Garfield': 2,
         'Wilson': 2})

In [156]:
(Counter(cluster_labels_df[cluster_labels_df["km_clusters"] == 6]["speech_source"]))

Counter({'McKinley': 3,
         'Garfield': 4,
         'Wilson': 4,
         'Obama': 1,
         'Roosevelt': 2,
         'Clinton': 6,
         'Nixon': 1})

In [158]:
(Counter(cluster_labels_df[cluster_labels_df["km_clusters"] == 5]["speech_source"]))

Counter({'Wilson': 3,
         'Nixon': 4,
         'Eisenhower': 11,
         'Cleveland': 12,
         'Garfield': 3,
         'Reagan': 3,
         'Roosevelt': 4,
         'McKinley': 7,
         'Clinton': 11,
         'Obama': 3})

In [159]:
(Counter(cluster_labels_df[cluster_labels_df["km_clusters"] == 0]["speech_source"]))

Counter({'Roosevelt': 76,
         'McKinley': 84,
         'Clinton': 66,
         'Wilson': 55,
         'Eisenhower': 79,
         'Reagan': 55,
         'Obama': 48,
         'Nixon': 47,
         'Garfield': 44,
         'Cleveland': 25})

In [161]:
(Counter(cluster_labels_df[cluster_labels_df["sc_clusters"] == 4]["speech_source"]))

Counter({'Nixon': 5,
         'Eisenhower': 31,
         'Roosevelt': 11,
         'Cleveland': 13,
         'McKinley': 25,
         'Reagan': 8,
         'Wilson': 10,
         'Clinton': 3,
         'Garfield': 3})

### Running k-means and spectral clustering with 2 and 3 clusters 
#### Hypothesis: Might cluster by time period or political party

**In Chronological Order:**
1. Obama
2. Clinton
3. Reagan
4. Nixon
5. Eisenhower
6. Roosevelt
7. Wilson
8. McKinley
9. Cleveland
10. Garfield

**Political Party Affiliation:**

Republican
+ Garfield
+ McKinley
+ Eisenhower
+ Nixon
+ Reagan

Democrat
+ Cleveland
+ Roosevelt
+ Wilson
+ Obama
+ Clinton

In [162]:
kmeans3 = KMeans(n_clusters=3)
kmeans3 = kmeans3.fit(X_train)
km3labels = kmeans3.predict(X_train)

In [163]:
Counter(km3labels)

Counter({0: 862, 1: 130, 2: 137})

In [164]:
kmeans2 = KMeans(n_clusters=2)
kmeans2 = kmeans2.fit(X_train)
km2labels = kmeans2.predict(X_train)

In [165]:
Counter(km2labels)

Counter({0: 986, 1: 143})

In [166]:
len(km2labels)

1129

In [168]:
sc3 = SpectralClustering(n_clusters=3)
sc3 = sc3.fit(X_train)
sc3labels = sc3.labels_

Counter(sc3labels)

Counter({0: 933, 2: 186, 1: 10})

In [170]:
sc2 = SpectralClustering(n_clusters=2)
sc2 = sc2.fit(X_train)
sc2labels = sc2.labels_

Counter(sc2labels)

Counter({0: 1117, 1: 12})

In [171]:
cluster_labels_df["km2_clusters"] = km2labels
cluster_labels_df["km3_clusters"] = km3labels


In [172]:
cluster_labels_df.head()

,speech_source,km_clusters,ms_clusters,sc_clusters,km2_clusters,km3_clusters
1397,Roosevelt,0,0,0,0,0
711,McKinley,0,0,0,0,0
247,Clinton,0,0,0,0,0
667,McKinley,0,0,0,0,0
1199,Wilson,0,0,0,0,0


### Digging into the different clustering methods (after running with 2 and 3 clusters)

In [173]:
(Counter(cluster_labels_df[cluster_labels_df["km2_clusters"] == 1]["speech_source"]))

Counter({'Wilson': 7,
         'Eisenhower': 23,
         'Cleveland': 30,
         'Nixon': 9,
         'McKinley': 24,
         'Clinton': 18,
         'Garfield': 7,
         'Roosevelt': 14,
         'Obama': 5,
         'Reagan': 6})